In [ ]:
%cd ..

# PreCode

In [ ]:
import os
from recfldgrn.datapoint import load_df_data_with_RecName, DataPoint
IDName = 'PID'; RANGE_SIZE = 10000

######################
rec_folder = 'data/ProcData/RecFolder'
####################

RecName_List = os.listdir(rec_folder)
print(RecName_List)
IDValue = 'P5'   # PhyID value
# print(data_folder)
Pat = DataPoint(IDName, IDValue, rec_folder, RANGE_SIZE)
# print(Pat)
dfx = Pat.get_df_rec('P')
df = load_df_data_with_RecName(rec_folder, 'P')
# dfx

# PNSect@SectTxt-TknzGrn

## df_whole/row

### Settings

In [ ]:
###################
RecName_Chain = ['P', 'EC', 'PNSect'] # this can be adjusted
RecTableName2FldColumns_Dict = {
    # The first should be in the level of RecName_Chain[-1]
    'EC': ['PID', 'ECID', 'DT_min', 'DT_max'], # prepare the datetime in advance. 
    'PN': ['PID', 'ECID', 'PNID', 'DT'],
    'PNSect': ['PID', 'ECID', 'PNSectID', 'SectName'],
    'PNSectSent': ['PID', 'ECID', 'PNSectID', 'PNSectSentID', 'Sentence'],
    # TODO: to add the df_whole filter or dp filter, you may want to add the EC DT information. 
}
###################

########################
## Prediction Index Date
RecName = RecName_Chain[-1] # row level
SynFld = 'PNSectTxt' # This need to be unique. 
GrnName = 'TknzGrn'
########################

###################
rec_folder = 'data/ProcData/RecFolder/'
###################

###################
# sfx_info = {'Cate': ['tkn', 'tpc'], 'Nume': ['wgt']}
reshape_fn_kwargs = {}
###################

### Load df_whole/row

In [ ]:
from recfldgrn.loadtools import get_df_whole_from_settings, get_df_individual_from_settings

recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
full_recfldgrn = f'{"-".join(RecName_Chain)}@{SynFld}-{GrnName}'

df_whole = get_df_whole_from_settings(RecName_Chain, RecTableName2FldColumns_Dict, rec_folder)
# df_whole = get_df_individual_from_settings(RecName_Chain,  RecTableName2FldColumns_Dict, Pat,)
row = df_whole.iloc[0]

### Filter df_whole / row (optional)

In [ ]:
import pandas as pd

for Rec in RecTableName2FldColumns_Dict:
    print(f'\n{Rec} =======')
    s = df_whole[Rec].apply(lambda x: len(x) if type(x) == type(pd.DataFrame()) else 'Missing')
    print(s.value_counts().sort_index())

## grain_tkn_fn

### Function Development

In [ ]:
from recfldgrn.graintools import func_convert_Num_factory
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
x = 'No section sentences' # * 500
tokenizer(x, truncation=True, max_length=512)

In [ ]:
li_value_tkn = tokenizer.tokenize(x, truncation=True, max_length=512, add_special_tokens=True)
# print(len(li_value_tkn))
li_value_tkn

In [ ]:
li_value_tknidx = tokenizer(x, truncation=True, max_length=512)['input_ids']
# li_value_tknidx

In [ ]:
v2idx = tokenizer.get_vocab()

li_value_tknidx = [v2idx[i] for i in  li_value_tkn]
li_value_tknidx

In [ ]:
# df_whole

In [ ]:
import inspect

def grain_tkn_fn(row):
    # please notice here that the information within dp is already filtered.
    
    # (0) Set up.
    import itertools
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    max_length = 512
    used_leng = 0 # how many token positions used so far. 
    buffer_dict = {}
    
    # (A) PNSect Part: 
    df = row['PNSect']
    # (A.1) PNSect@SectName
    fld = 'SectName'
    if len(df) > 0:
        df_recinput = df[['SectName']].fillna('No section sentences')
        value = df_recinput[fld].iloc[0] # section name
    else:
        fld = 'SectName'
        value = 'No sections'
        
    current_max_leng = max_length - used_leng # new line
    li_value_tkn = tokenizer.tokenize(value, truncation=True, max_length=current_max_leng, add_special_tokens=True)
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    used_leng = len(li_value_tkn)   # new line
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # (B) PNSectSent Part
    df = row['PNSectSent']
    # (B.1) PN-PNSectSent
    fld = 'Sentence'
    if len(df) > 0:
        df_recinput = df[['Sentence']].fillna('No section sentences')
        value = '\n'.join(df_recinput['Sentence'].to_list())
    else:
        value = 'No section sentences'
    current_max_leng = max_length - used_leng # new line
    li_value_tkn = tokenizer.tokenize(value, truncation=True, max_length=current_max_leng, add_special_tokens=True)
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    used_leng = len(li_value_tkn)   # new line
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # summary
    output = {}
    for i in ['key', 'wgt', 'tpc']:
        output[i.split('_')[-1]] = list(itertools.chain(*[d[i] for fld, d in buffer_dict.items()]))
    return output

grain_tkn_fn.fn_string = inspect.getsource(grain_tkn_fn)
# print(grain_str_fn_string)

### Test

In [ ]:
output = grain_tkn_fn(row)
for name, grn_list in output.items():
    print('\n')
    print(name)
    print(grn_list)
    print(len(grn_list))

In [ ]:
recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
df_whole[recfldgrn] = df_whole.apply(lambda row: grain_tkn_fn(row), axis = 1)
df_fld = df_whole[[R+'ID' for R in RecName_Chain] + [recfldgrn]]
df_fld

## Save PyFile

### Prepare Variables

In [ ]:
string_variables = [RecName, SynFld, GrnName, rec_folder]
iterative_variables = [RecName_Chain, RecTableName2FldColumns_Dict, reshape_fn_kwargs]
fn_variables = [grain_tkn_fn]
prefix = ['import pandas as pd', 'import numpy as np', 'import itertools']

### Save to PyFile

In [ ]:
from recfldgrn.loadtools import retrieve_name, convert_variables_to_pystirng

python_strings  = convert_variables_to_pystirng(string_variables, iterative_variables, fn_variables, prefix)
Meta_folder = rec_folder.replace('RecFolder', 'FldGrnMeta')
meta_file = os.path.join(Meta_folder, SynFld + '.py')
print(meta_file)
with open(meta_file, 'w') as f: 
    f.write(python_strings)

# Application

In [ ]:
%cd ..

## Settings

In [ ]:
##########
rec_folder = 'data/ProcData/RecFolder/'
SynFld = 'PNSectTxt' # ----> get its MetaDict
##########


# filtering hyperparameters
######### get df_whole: obsind, predind, and futind
PpredTable = 'xxx'
PpredColum = 'xxx'
Range_List = ['Bf2y']
#########

## Load FldGrn PyFile

In [ ]:
import os
from recfldgrn.loadtools import load_module_variables

Meta_folder = rec_folder.replace('RecFolder', 'FldGrnMeta')
meta_file = os.path.join(Meta_folder, SynFld + '.py')
module = load_module_variables(meta_file)
MetaDict = module.MetaDict

RecName = MetaDict['RecName']#  'P'
RecName_Chain = MetaDict['RecName_Chain'] # ['P']
# RecName_Chain_to_RecName = MetaDict['RecName_Chain_to_RecName'] # {} # Can be updated further. 
RecTableName2FldColumns_Dict = MetaDict['RecTableName2FldColumns_Dict']
# SynFld = MetaDict['SynFld'] # 'PECINfo'
GrnName = MetaDict['GrnName'] # 'InfoGrn'
# sfx_info = MetaDict['sfx_info']
# rec_folder = MetaDict['rec_folder'] # 'data/ProcData/RecFolder/'
reshape_fn_kwargs = MetaDict['reshape_fn_kwargs']
grain_tkn_fn = MetaDict['grain_tkn_fn']

recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
recfldgrn

## Load & Filter df_whole/row

In [ ]:
RecName_Chain = RecName_Chain # only adjust
RecTableName2FldColumns_Dict = RecTableName2FldColumns_Dict # TODO, to update.
# RecTableName2FldColumns_Dict[PpredTable] = [PpredColum]

In [ ]:
from recfldgrn.loadtools import get_df_whole_from_settings, get_df_individual_from_settings
df_whole = get_df_whole_from_settings(RecName_Chain, RecTableName2FldColumns_Dict,  rec_folder)

In [ ]:
## filtering

## Generate RecFldGrn

In [ ]:
recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
df_whole[recfldgrn] = df_whole.apply(lambda row: grain_tkn_fn(row), axis = 1)
df_fld = df_whole[[R+'ID' for R in RecName_Chain] + [recfldgrn]].reset_index(drop = True)
df_fld

In [ ]:
df_fld['PNSectID'].value_counts()

## Save FldGrn Data (Optional)